# UDP Encryption and Decryption with Two Alveo U280s in OCT 

In [1]:
import time

## Sender/Receiver Parameters

In [2]:
remote_host_sender = '<node ID>.cloudlab.umass.edu' #eg: pc155.cloudlab.umass.edu
remote_host_receiver = '<node ID>.cloudlab.umass.edu'
remote_host_username = '<CloudLab username>'
private_key = '<private key>'
dest_dir = '/users/' + remote_host_username + '/udp-demo'

In [3]:
bitstream = 'udp_demo_if0.xclbin'
num_packets = 1
encrypt = True # (False if encryption is not required)
decrypt = True # (False if decryption is not required)
sender_ip = '192.168.40.100'
receiver_ip = '192.168.40.200'
ip_gateway = '192.168.40.254'

## Check the SSH Connection to the FPGA Nodes in OCT

In [4]:
!ssh -o StrictHostKeyChecking=no -i {private_key} {remote_host_username}@{remote_host_sender} "echo \"Successfully connected to {remote_host_sender}\" && mkdir -p {dest_dir}"

Successfully connected to pc151.cloudlab.umass.edu


In [5]:
!ssh -o StrictHostKeyChecking=no -i {private_key} {remote_host_username}@{remote_host_receiver} "echo \"Successfully connected to {remote_host_receiver}\" && mkdir -p {dest_dir}"

Successfully connected to pc154.cloudlab.umass.edu


## Copy Files

In [6]:
!scp -i {private_key} ../build_hw_if0/udp_demo_if0.xclbin sender.py validate.py vnx_utils.py ../host/alice29.txt {remote_host_username}@{remote_host_sender}:{dest_dir}

udp_demo_if0.xclbin                           100%   53MB 105.3MB/s   00:00    
sender.py                                     100% 2241   497.4KB/s   00:00    
validate.py                                   100%  413    93.1KB/s   00:00    
vnx_utils.py                                  100%   32KB   6.3MB/s   00:00    
alice29.txt                                   100%  149KB   3.4MB/s   00:00    


In [7]:
!scp -i {private_key} ../build_hw_if0/udp_demo_if0.xclbin receiver.py validate.py vnx_utils.py {remote_host_username}@{remote_host_receiver}:{dest_dir}

udp_demo_if0.xclbin                           100%   53MB  99.3MB/s   00:00    
receiver.py                                   100% 2063   248.1KB/s   00:00    
validate.py                                   100%  413    90.4KB/s   00:00    
vnx_utils.py                                  100%   32KB   6.1MB/s   00:00    


## Sender/Receiver Commands

In [8]:
dest_dir_cmd = f"cd {dest_dir};"
# Rx command
xrt_cmd = 'source /opt/xilinx/xrt/setup.sh;'
rx_params = f"{bitstream} --num_packets {num_packets} --decrypt {decrypt} --my_ip {receiver_ip} --their_ip {sender_ip} --ip_gw {ip_gateway}"
rx_run_cmd = f"python3.8 receiver.py {rx_params}"
rx_buf_del = 'rm -f rx_buf.txt;'
rx_cmd = 'nohup /bin/bash -c ' + f"'{dest_dir_cmd} {rx_buf_del} {xrt_cmd} {rx_run_cmd}'" + f" > {dest_dir}/rx_out.log 2>&1 &"
# Tx command
tx_params = f"{bitstream} --num_packets {num_packets} --encrypt {encrypt} --my_ip {sender_ip} --their_ip {receiver_ip} --ip_gw {ip_gateway}"
tx_run_cmd = f"python3.8 sender.py {tx_params}"
tx_cmd = 'nohup /bin/bash -c ' + f"'{dest_dir_cmd} {xrt_cmd} {tx_run_cmd}'" + f" > {dest_dir}/tx_out.log 2>&1 &"

### Receiver

In [9]:
!ssh -i {private_key} {remote_host_username}@{remote_host_receiver} "{rx_cmd}"

### Sender

In [10]:
!ssh -i {private_key} {remote_host_username}@{remote_host_sender} "{tx_cmd}"

In [11]:
time.sleep(5)

## Retrieve the Textfile and Display

In [12]:
!rm -f ./rx_buf.txt
!scp -i {private_key} {remote_host_username}@{remote_host_receiver}:{dest_dir}/rx_buf.txt .

rx_buf.txt                                    100% 1408   321.1KB/s   00:00    


In [13]:
file_path = './rx_buf.txt'
with open(file_path, 'r') as file:
    content = file.read()
print(content)





                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.9




                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `without pictures or conversation?'

  So she was considering in her own mind (as well as she could,
for the hot day made her feel very sleepy and stupid), whether
the pleasure of making a daisy-chain would be worth the trouble
of getting up and picking the daisies, when suddenly a White
Rabbit with pink eyes ran close by her.

  There was nothing so VERY remarkable in that; nor did Alice
think it so VERY much out of the way to hear the Rabbit say to
itself, `Oh dear!  Oh dear!  I shall be lat